# Coastal Erosion Simulation: Finding the Initial Parameters

## Introduction
This notebook tries to find reasonable estimates for parameters modeling the simulation world of the Norfolk coastline.

The simulation incorporates data from NOAA buoy **62144**, alongside empirical equations derived from relevant literature, to calculate wave and coastal parameters such as wave height, wave speed, sand pull, ground pull, and others. Below is an explanation of the steps, models, and equations used in this analysis.

#### Sources:

https://www.ndbc.noaa.gov/
Environment Agency, Sea State Report: Felixstowe Wave Buoy (2013): https://www.coastalmonitoring.org/pdf_download/?metadata_id=486318

Bouws, E., & Komen, G. J. (1983). On the balance between growth and dissipation in an extreme depth-limited wind-sea in the southern North Sea. Journal of physical oceanography, 13(9), 1653-1658.

Walkden, M. J. A., & Hall, J. W. (2005). A predictive mesoscale model of the erosion and profile development of soft rock shores. Coastal Engineering, 52(6), 535-563.

Bruun, P. (1962). Sea-level rise as a cause of shore erosion. Journal of the Waterways and Harbors division, 88(1), 117-130.

## 1. Fetching Buoy Data
The simulation starts by fetching real-time wave data from NOAA's database for the buoy nearest Norfolk. Key parameters such as **wave height** and **wave speed** are extracted and averaged.

In [10]:
import pandas as pd

# Fetch real-time data for the closest buoy
buoy_id = 62144
data_url = f"https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.txt"

# Load buoy data
try:
    data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])
    print(data.head())
except Exception as e:
    print(f"Error fetching data for buoy {buoy_id}: {e}")

C:\Users\sonia\AppData\Local\Temp\ipykernel_24980\896513729.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])


    #YY  MM  DD  hh  mm  WDIR  WSPD  GST  WVHT  DPD  APD  MWD    PRES  ATMP  \
0  2025   1  18  23   0   150   2.1  NaN   0.5  NaN  NaN  NaN  1026.0   3.9   
1  2025   1  18  22   0   150   2.1  NaN   0.5  NaN  NaN  NaN  1026.2   3.7   
2  2025   1  18  21   0   150   1.5  NaN   0.5  NaN  NaN  NaN  1027.0   3.7   
3  2025   1  18  20   0   150   2.1  NaN   0.5  NaN  NaN  NaN  1027.2   3.6   
4  2025   1  18  19   0   150   2.1  NaN   0.5  NaN  NaN  NaN  1027.7   3.6   

   WTMP  DEWP  VIS  PTDY  TIDE  
0   NaN   2.8   11  -1.2   NaN  
1   NaN   2.8   11  -1.4   NaN  
2   NaN   3.0    5  -1.0   NaN  
3   NaN   2.9   11  -1.1   NaN  
4   NaN   3.1   11  -0.9   NaN  


Extract the relevant data

In [11]:
# Drop rows with missing wave height
data = data.dropna(subset=["WVHT"])

wave_height = data["WVHT"].mean()  # Significant wave height
wave_speed = data["WSPD"].mean()  

# Output parameters
print(f"WAVE_HEIGHT: {wave_height}") # m
print(f"WAVE_SPEED: {wave_speed}") # m/s

WAVE_HEIGHT: 0.528
WAVE_SPEED: 3.52


## 2. Estimating the Unknown

Using models from the literature, empirical findings, and some approximations.

In [13]:
import numpy as np

# Constants
g = 9.81  # gravity (m/s^2)
rho = 1025  # water density (kg/m^3)
T = 4.81  # representative wave period (s)
H_s = 0.564  # significant wave height (m)
c = 4.088  # wave speed (m/s)
W = 1  # wave front width (m), assumed for 1 m segment
K = 1.8  # wave retreat coefficient (calibrated for Norfolk)
R = 1.27e6  # material resistance (N/m^2), assumed based on the model in the report
f1 = 0.8  # wave breaking term (estimated based on report)
f2 = 1  # slope constant
z = 0.05  # shore platform slope (radians) from the report
f3_t = 1  # time-dependent tidal influence, assumed constant for simplicity
theta = np.radians(12)  # wave approach angle in radians (12 degrees from the report)

# 1. Wave Frequency
wave_freq = 1 / T

# 2. Wave Decay (empirical coefficient)
wave_decay = 0.038  # Empirical coefficient from JONSWAP (from the report)

# 3. Wave Cutoff Depth (based on report, as a fraction of H_s)
wave_cutoff = 0.23 * H_s  # Cutoff depth based on H_s

# 4. Wave Retreat Coefficient (direct usage of K as per report)
wave_retreat_coeff = K  # Retreat rate in m/year (using 1.8 directly)

# 5. Sand Pull (CERC Equation from the report)
H_b = H_s  # Assuming breaking wave height is the same as significant wave height
sand_pull = K * H_b**(5/2) * np.sin(2 * theta)  # Sand pull in m³/year (CERC model)

# 6. Ground Pull (Erosion Rate Model from the report)
F = rho * g * H_s**2  # Wave force in N/m² (calculated from wave height)
ground_pull = (F / R) * f1 * f3_t * np.tan(f2 * z)  # Erosion rate in m/year (calculated)

# 7. Wave Volume (calculated from wave height and wavelength)
wavelength = (g * T**2) / (2 * np.pi)  # Wavelength using deep water wave formula
wave_volume = (H_s / 2) * wavelength * W  # Wave volume in m³/wave

# 8. Wave Spread (as per the report, fixed at 12 degrees)
wave_spread = 12  # degrees

# Output results with updated precision
print(f"Wave Frequency: {wave_freq:.6f} Hz")
print(f"Wave Decay: {wave_decay}")
print(f"Wave Cutoff Depth: {wave_cutoff:.6f} m")
print(f"Wave Retreat Coefficient: {wave_retreat_coeff}")
print(f"Sand Pull: {sand_pull:.6f} m³/year")
print(f"Ground Pull: {ground_pull:.6f} m/year")
print(f"Wave Spread: {wave_spread} degrees")
print(f"Wave Volume: {wave_volume:.6f} m³/wave")

Wave Frequency: 0.207900 Hz
Wave Decay: 0.038
Wave Cutoff Depth: 0.129720 m
Wave Retreat Coefficient: 1.8
Sand Pull: 0.174897 m³/year
Ground Pull: 0.000101 m/year
Wave Spread: 12 degrees
Wave Volume: 10.186580 m³/wave
